In [1]:
# ===============================================
# SPEP Quantification Tool – Named Lanes & Bands,
# Manual Control Entry, Dual View, Lazy NumPy,
# Notebook Output + Save-as-CSV
# ===============================================

import os, math, statistics
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox, ttk

# ---------------- CONFIG ----------------
IMAGE_PATH = ""        # leave empty to open picker
SAVE_PREFIX = "SPEP_"  # still used by "save all" (optional)
DEFAULT_BANDS = ["Albumin", "Alpha1", "Alpha2", "Beta", "Gamma"]

# Regression/Output options
FIT_THRU_ORIGIN = False   # True => force y = a*x (no intercept)
CLAMP_NONNEG    = True    # True => clamp predicted g/dL to >= 0
# ----------------------------------------

# Lazy NumPy importer (for regression only)
def lazy_np():
    try:
        import numpy as _np
        return _np
    except Exception:
        return None

# ---------- Data classes ----------
@dataclass
class Rect:
    x: float; y: float; w: float; h: float

@dataclass
class Lane:
    lane_id: str
    lane_type: str  # "NEG","POS","SAMPLE"
    name: str
    rect: Rect

@dataclass
class Band:
    lane_id: str
    band_name: str
    rect: Rect

# ---------- Image helpers ----------
def load_image_gray(path: str) -> Image.Image:
    img = Image.open(path)
    if img.mode != "L":
        img = img.convert("L")
    return img

def crop(img: Image.Image, r: Rect) -> Image.Image:
    x0 = max(0, int(round(r.x))); y0 = max(0, int(round(r.y)))
    x1 = min(img.width, int(round(r.x+r.w))); y1 = min(img.height, int(round(r.y+r.h)))
    return img.crop((x0, y0, x1, y1))

def iod_intensity(roi: Image.Image) -> float:
    # invert (dark bands -> high signal), background = median of 10% border, sum positive
    inv = Image.eval(roi, lambda p: 255 - p)
    w, h = inv.size
    px = list(inv.getdata())
    b = max(1, int(0.1 * min(w, h)))
    top = px[:w*b]
    bot = px[-w*b:] if h > b else []
    left, right = [], []
    for yy in range(h):
        row = yy*w
        left.extend(px[row:row+b])
        right.extend(px[row+(w-b):row+w])
    border = top + bot + left + right
    bg = statistics.median(border) if border else 0
    return float(sum(max(v-bg, 0) for v in px))

# ---------- Tk utilities ----------
def _fit_scale(img: Image.Image, max_w: int, max_h: int) -> Tuple[Image.Image, float]:
    scale = min(max_w/img.width, max_h/img.height, 1.0)
    return (img.resize((int(img.width*scale), int(img.height*scale)), Image.BILINEAR), scale)

# Lane selector (with lane naming)
class LaneSelector:
    def __init__(self, img: Image.Image, label="LANES"):
        self.img = img
        self.root = tk.Tk()
        self.root.title(f"Select {label} — Enter/Esc=Done, u=Undo")
        scr_w = self.root.winfo_screenwidth(); scr_h = self.root.winfo_screenheight()
        disp, self.scale = _fit_scale(img, int(scr_w*0.9), int(scr_h*0.85))
        self.tk_img = ImageTk.PhotoImage(disp)
        self.canvas = tk.Canvas(self.root, width=disp.width, height=disp.height, cursor="cross")
        self.canvas.grid(row=0, column=0, sticky="nsew")
        self.root.grid_rowconfigure(0, weight=1); self.root.grid_columnconfigure(0, weight=1)
        self.canvas.create_image(0,0,image=self.tk_img,anchor="nw")
        self.canvas.create_text(10,10,anchor="nw",
            text="Drag to add lane rectangles. You'll be asked to name each.",
            fill="lime", font=("Helvetica",12,"bold"))
        btn = tk.Frame(self.root); btn.grid(row=1,column=0,sticky="ew")
        tk.Button(btn,text="Undo",command=self.undo).pack(side=tk.LEFT,padx=6,pady=6)
        tk.Button(btn,text="Done",command=self.finish).pack(side=tk.RIGHT,padx=6,pady=6)
        self.root.bind("<Return>",lambda e:self.finish())
        self.root.bind("<Escape>",lambda e:self.finish())
        self.root.bind("u",lambda e:self.undo())
        self.root.bind("<BackSpace>",lambda e:self.undo())
        self.start=None; self._curr=None
        self.can_ids=[]; self.rects=[]; self.names=[]
        self.canvas.bind("<ButtonPress-1>",self.on_press)
        self.canvas.bind("<B1-Motion>",self.on_drag)
        self.canvas.bind("<ButtonRelease-1>",self.on_release)
    def on_press(self,e):
        self.start=(e.x,e.y)
        self._curr=self.canvas.create_rectangle(e.x,e.y,e.x,e.y,outline="lime",width=2)
    def on_drag(self,e):
        if self._curr and self.start:
            x0,y0=self.start; self.canvas.coords(self._curr,x0,y0,e.x,e.y)
    def on_release(self,e):
        if not self._curr or not self.start: return
        x0,y0=self.start; x1,y1=e.x,e.y
        x,y=min(x0,x1),min(y0,y1); w,h=abs(x1-x0),abs(y1-y0)
        if w>=2 and h>=2:
            self.can_ids.append(self._curr)
            rx,ry=x/self.scale,y/self.scale; rw,rh=w/self.scale,h/self.scale
            self.rects.append(Rect(rx,ry,rw,rh))
            nm=simpledialog.askstring("Lane name","Name for this lane:",parent=self.root)
            if not nm: nm=f"Lane-{len(self.rects)}"
            self.names.append(nm)
            self.canvas.create_text(x+4,y+14,anchor="nw",text=nm,fill="yellow",font=("Helvetica",11,"bold"))
        else:
            self.canvas.delete(self._curr)
        self._curr=None; self.start=None
    def undo(self):
        if self.can_ids:
            self.canvas.delete(self.can_ids.pop()); self.rects.pop(); self.names.pop()
    def finish(self):
        try:self.root.destroy()
        except:pass
    def select(self)->List[Tuple[Rect,str]]:
        self.root.mainloop()
        return list(zip(self.rects,self.names))

# Dual-view band selector (full SPEP + zoomed lane)
class BandSelectorDual:
    def __init__(self, full_img: Image.Image, lane_rect: Rect, title="Select bands"):
        self.full=full_img; self.lane_rect=lane_rect
        self.root=tk.Tk(); self.root.title(title)
        scr_w=self.root.winfo_screenwidth(); scr_h=self.root.winfo_screenheight()
        left_w=int(scr_w*0.45); right_w=int(scr_w*0.45); max_h=int(scr_h*0.9)
        dispL,self.scaleL=_fit_scale(self.full,left_w,max_h)
        self.tkL=ImageTk.PhotoImage(dispL)
        self.canvasL=tk.Canvas(self.root,width=dispL.width,height=dispL.height)
        self.canvasL.grid(row=0,column=0,sticky="nsew",padx=(8,4),pady=8)
        self.canvasL.create_image(0,0,image=self.tkL,anchor="nw")
        x,y,w,h=lane_rect.x*self.scaleL,lane_rect.y*self.scaleL,lane_rect.w*self.scaleL,lane_rect.h*self.scaleL
        self.canvasL.create_rectangle(x,y,x+w,y+h,outline="deepskyblue",width=3)
        lane_img=crop(self.full,lane_rect)
        dispR,self.scaleR=_fit_scale(lane_img,right_w,max_h)
        self.tkR=ImageTk.PhotoImage(dispR)
        self.canvasR=tk.Canvas(self.root,width=dispR.width,height=dispR.height,cursor="cross")
        self.canvasR.grid(row=0,column=1,sticky="nsew",padx=(4,8),pady=8)
        self.canvasR.create_image(0,0,image=self.tkR,anchor="nw")
        self.canvasR.create_text(8,10,anchor="nw",
            text="Draw band rectangles (top→bottom). Enter/Esc=Done, u=Undo",
            fill="lime",font=("Helvetica",12,"bold"))
        self.root.grid_columnconfigure(0,weight=1);self.root.grid_columnconfigure(1,weight=1)
        self.root.grid_rowconfigure(0,weight=1)
        btn=tk.Frame(self.root);btn.grid(row=1,column=0,columnspan=2,sticky="ew")
        tk.Button(btn,text="Undo",command=self.undo).pack(side=tk.LEFT,padx=6,pady=6)
        tk.Button(btn,text="Done",command=self.finish).pack(side=tk.RIGHT,padx=6,pady=6)
        self.root.bind("<Return>",lambda e:self.finish())
        self.root.bind("<Escape>",lambda e:self.finish())
        self.root.bind("u",lambda e:self.undo())
        self.root.bind("<BackSpace>",lambda e:self.undo())
        self.start=None;self._curr=None;self.can_ids=[];self.rects_local=[]
        self.canvasR.bind("<ButtonPress-1>",self.on_press)
        self.canvasR.bind("<B1-Motion>",self.on_drag)
        self.canvasR.bind("<ButtonRelease-1>",self.on_release)
    def on_press(self,e):
        self.start=(e.x,e.y)
        self._curr=self.canvasR.create_rectangle(e.x,e.y,e.x,e.y,outline="lime",width=2)
    def on_drag(self,e):
        if self._curr and self.start:
            x0,y0=self.start;self.canvasR.coords(self._curr,x0,y0,e.x,e.y)
    def on_release(self,e):
        if not self._curr or not self.start:return
        x0,y0=self.start;x1,y1=e.x,e.y
        x,y=min(x0,x1),min(y0,y1);w,h=abs(x1-x0),abs(y1-y0)
        if w>=2 and h>=2:
            self.can_ids.append(self._curr)
            rx,ry=x/self.scaleR,y/self.scaleR;rw,rh=w/self.scaleR,h/self.scaleR
            self.rects_local.append(Rect(rx,ry,rw,rh))
        else:self.canvasR.delete(self._curr)
        self._curr=None;self.start=None
    def undo(self):
        if self.can_ids:
            self.canvasR.delete(self.can_ids.pop());self.rects_local.pop()
    def finish(self):
        try:self.root.destroy()
        except:pass
    def select(self)->List[Rect]:
        self.root.mainloop();return self.rects_local

# ---------- Selection helpers ----------
def select_lanes(img: Image.Image, label: str) -> List[Lane]:
    ls=LaneSelector(img,label=f"{label} LANES")
    rects_and_names=ls.select()
    return [Lane(lane_id=f"{label}-{i+1}", lane_type=label, name=n, rect=r)
            for i,(r,n) in enumerate(rects_and_names)]

def select_bands(img: Image.Image, lane: Lane) -> List[Rect]:
    bs=BandSelectorDual(img,lane.rect,title=f"{lane.lane_type} / {lane.name}")
    return bs.select()

def name_control_bands(n:int,preset:Optional[List[str]]=None)->List[str]:
    names=preset[:] if preset else (DEFAULT_BANDS[:n] if n==5 else [f"Band{i+1}" for i in range(n)])
    out=[]
    for i in range(n):
        nm=simpledialog.askstring("Band name",f"Name for control band #{i+1}:",initialvalue=names[i])
        out.append(nm if nm else names[i])
    return out

def prompt_control_concs(lane_name:str, band_names:List[str])->Dict[str,float]:
    concs={}
    for bn in band_names:
        while True:
            val=simpledialog.askstring("Control concentration",f"{lane_name} — {bn} (g/dL):")
            if val is None or val.strip()=="":
                if messagebox.askyesno("Skip?","Blank input. Skip this band?"): break
                else: continue
            try: concs[bn]=float(val);break
            except: messagebox.showerror("Invalid","Please enter a number.")
    return concs

# ---------- Calibration / Prediction ----------
def linear_fit(x_list, y_list, through_origin=False):
    np = lazy_np()
    if np is not None:
        x = np.asarray(x_list, float); y = np.asarray(y_list, float)
        if through_origin:
            a = (x @ y) / max((x @ x), 1e-12); b = 0.0
        else:
            A = np.vstack([x, np.ones_like(x)]).T
            a, b = np.linalg.lstsq(A, y, rcond=None)[0]
        return float(a), float(b)
    # pure-Python fallback
    if through_origin:
        sxx = sum(v*v for v in x_list)
        sxy = sum(x*y for x,y in zip(x_list,y_list))
        a = sxy / (sxx + 1e-12); b = 0.0
        return a, b
    n=len(x_list); sx=sum(x_list); sy=sum(y_list)
    sxx=sum(v*v for v in x_list)
    sxy=sum(x*y for x,y in zip(x_list,y_list))
    denom = n*sxx - sx*sx
    if abs(denom) < 1e-12:
        a = sxy/(sxx+1e-12); b = 0.0
    else:
        a = (n*sxy - sx*sy)/denom; b = (sy - a*sx)/n
    return a, b

def calibrate(measured: List[Dict], controls: List[Dict]) -> Tuple[Dict[str,Tuple[float,float]], List[Dict]]:
    meas = {(r["lane_id"], r["band_name"]): r["iod_intensity"] for r in measured}
    by_band = {}
    for c in controls:
        key = (c["lane_id"], c["band_name"])
        if key in meas and c.get("conc_g_dl") is not None:
            by_band.setdefault(c["band_name"], []).append((meas[key], float(c["conc_g_dl"]), c["lane_id"]))
    models, diag = {}, []
    for band, triples in by_band.items():
        xs=[t[0] for t in triples]; ys=[t[1] for t in triples]; lids=[t[2] for t in triples]
        a,b = linear_fit(xs, ys, through_origin=FIT_THRU_ORIGIN)
        models[band]=(a,b)
        for xi,yi,lid in zip(xs,ys,lids):
            pred = a*xi + b
            diag.append(dict(band_name=band, lane_id=lid, iod_intensity=xi, conc_g_dl=yi, pred=pred, resid=yi-pred))
    return models, diag

def predict(measured: List[Dict], models: Dict[str,Tuple[float,float]], sample_ids: List[str],
            lane_name_map: Dict[str,str]) -> List[Dict]:
    out=[]
    for r in measured:
        if r["lane_id"] in sample_ids:
            a,b = models.get(r["band_name"], (float("nan"), float("nan")))
            pred = a*r["iod_intensity"] + b if not (math.isnan(a) or math.isnan(b)) else float("nan")
            if CLAMP_NONNEG and not math.isnan(pred):
                pred = max(0.0, pred)
            out.append(dict(
                lane_id=r["lane_id"],
                lane_name=lane_name_map.get(r["lane_id"], r["lane_id"]),
                band_name=r["band_name"],
                iod_intensity=r["iod_intensity"],
                conc_g_dl_est=pred
            ))
    return out

# ---------- CSV helper ----------
def write_csv(path:str, rows:List[Dict], header:Optional[List[str]]=None):
    import csv
    if not rows and not header: return
    if header is None: header = list(rows[0].keys())
    with open(path,"w",newline="") as f:
        w = csv.DictWriter(f, fieldnames=header); w.writeheader()
        for r in rows: w.writerow(r)

# ---------- Notebook table printer ----------
def print_table(rows: List[Dict]):
    if not rows:
        print("(no rows)"); return
    cols = ["Lane", "Band", "IOD", "Conc (g/dL)"]
    w_lane = max(len(cols[0]), max(len(r["lane_name"]) for r in rows))
    w_band = max(len(cols[1]), max(len(r["band_name"]) for r in rows))
    w_iod  = len(cols[2]); w_pred = len(cols[3])
    print(f"{cols[0]:<{w_lane}}  {cols[1]:<{w_band}}  {cols[2]:>{w_iod}}  {cols[3]:>{w_pred}}")
    print("-"*(w_lane+w_band+w_iod+w_pred+6))
    for r in rows:
        iod = f"{r['iod_intensity']:.0f}"
        pred = "NA" if math.isnan(r["conc_g_dl_est"]) else f"{r['conc_g_dl_est']:.3f}"
        print(f"{r['lane_name']:<{w_lane}}  {r['band_name']:<{w_band}}  {iod:>{w_iod}}  {pred:>{w_pred}}")

# ================ MAIN WORKFLOW ================
# 1) pick image
if not IMAGE_PATH:
    root=tk.Tk();root.withdraw()
    IMAGE_PATH=filedialog.askopenfilename(title="Choose gel image",
                                          filetypes=[("Images","*.jpg *.jpeg *.png *.tif *.tiff")])
    root.destroy()
if not IMAGE_PATH: raise SystemExit("No image selected.")
img=load_image_gray(IMAGE_PATH)

# 2) lanes (you name each)
neg_lanes=select_lanes(img,"NEG")
pos_lanes=select_lanes(img,"POS")
sample_lanes=select_lanes(img,"SAMPLE")
all_lanes=neg_lanes+pos_lanes+sample_lanes
if not all_lanes: raise SystemExit("No lanes selected.")

# 3) bands – name on first control, reuse for all
band_names_master=None
all_bands=[]

def collect_for_lane(lane: Lane, need_names: bool) -> List[Band]:
    global band_names_master
    rects = select_bands(img, lane)
    if need_names:
        band_names_master = name_control_bands(
            len(rects),
            preset=DEFAULT_BANDS if len(rects) == 5 else None
        )
        names = band_names_master
    else:
        names = (
            band_names_master
            if (band_names_master and len(band_names_master) == len(rects))
            else [f"Band{i+1}" for i in range(len(rects))]
        )
    bands=[]
    for nm, r in zip(names, rects):
        bands.append(Band(lane_id=lane.lane_id,
                          band_name=nm,
                          rect=Rect(r.x + lane.rect.x, r.y + lane.rect.y, r.w, r.h)))
    return bands

# controls first (name once)
first_control_done=False
for lane in (neg_lanes + pos_lanes):
    blist = collect_for_lane(lane, need_names=(not first_control_done))
    if not first_control_done: first_control_done=True
    all_bands.extend(blist)

# then samples (reuse names)
for lane in sample_lanes:
    all_bands.extend(collect_for_lane(lane, need_names=False))

# 4) measure
measured=[]
for b in all_bands:
    iod = iod_intensity(crop(img, b.rect))
    measured.append(dict(lane_id=b.lane_id, band_name=b.band_name, iod_intensity=iod))

# 5) interactive control concentrations (no CSV needed)
controls=[]
for lane in (neg_lanes + pos_lanes):
    band_list = band_names_master or sorted({r['band_name'] for r in measured if r['lane_id']==lane.lane_id})
    concs = prompt_control_concs(f"{lane.lane_type} / {lane.name}", band_list)
    for bn,val in concs.items():
        controls.append(dict(lane_id=lane.lane_id, band_name=bn, conc_g_dl=val))
if not controls: raise SystemExit("No control concentrations provided.")

# 6) calibrate & predict
models, diag = calibrate(measured, controls)
sample_ids = [l.lane_id for l in sample_lanes]
lane_name_map = {l.lane_id: l.name for l in all_lanes}
pred = predict(measured, models, sample_ids, lane_name_map)

# 7) show in notebook and ask to save
print("\nPredicted concentrations (g/dL)")
print_table(pred)

resp = input("\nSave predicted concentrations to CSV? [y/N]: ").strip().lower()
if resp == "y":
    fname = input("Filename (without extension) [default: sample_concentrations]: ").strip()
    if not fname: fname = "sample_concentrations"
    write_csv(fname + ".csv", pred,
              header=["lane_id","lane_name","band_name","iod_intensity","conc_g_dl_est"])
    print(f"Saved: {fname}.csv")

print("\nDone.")


2025-11-06 22:10:10.182 python[39130:21162412] +[IMKClient subclass]: chose IMKClient_Modern
2025-11-06 22:10:10.642 python[39130:21162412] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
2025-11-06 22:10:22.025 python[39130:21162412] +[IMKInputSession subclass]: chose IMKInputSession_Modern



Predicted concentrations (g/dL)
Lane  Band     IOD  Conc (g/dL)
-------------------------------
230   Albumin  181958        4.243
230   Alpha1   23814        0.244
230   Alpha2   11309        0.819
230   Beta     53852        0.597
230   Gamma    3180        0.338
321   Albumin  150529        3.506
321   Alpha1   14029        0.220
321   Alpha2   9998        0.822
321   Beta     72132        0.481
321   Gamma    1797        0.301
232   Albumin  166631        3.884
232   Alpha1   20301        0.236
232   Alpha2   10728        0.820
232   Beta     35540        0.712
232   Gamma    2116        0.310
233   Albumin  155244        3.617
233   Alpha1   17879        0.229
233   Alpha2   10650        0.820
233   Beta     48248        0.632
233   Gamma    1392        0.291



Save predicted concentrations to CSV? [y/N]:  y
Filename (without extension) [default: sample_concentrations]:  Sample5_SPEP


Saved: Sample5_SPEP.csv

Done.
